# BÀI THỰC HÀNH 3: MẠNG NEURAL HỒI QUY CHO BÀI TOÁN PHÂN LOẠI VĂN BẢN VÀ GÁN NHÃN CHUỖI

<b>Hướng dẫn nộp bài:</b> Các bạn commit và push code lên github, sử dụng file txt đặt tên theo cú pháp <MSSV>.txt chứa đường link dẫn đến github của bài thực hành và nộp file txt này tên courses.

Bộ dữ liệu sử dụng: [UIT-VSFC](https://drive.google.com/drive/folders/1rdcXNGt_3-QUvV8EtSvVsLMVeHmk6Yqk?usp=drive_link) và [PhoNERT](https://github.com/VinAIResearch/PhoNER_COVID19).

### Bài 1: Xây dựng mạng LSTM gồm 5 lớp với hidden size là 256 cho bài toán phân loại văn bản. Huấn luyện mô hình này trên bộ dữ liệu UIT-VSFC (Vietnamese Student Feedback Corpus) sử dụng Adam làm phương thức tối ưu tham số và đánh giá độ hiệu quả của mô hình sử dụng độ đo F1.

In [46]:
import tensorflow as tf
from tensorflow.keras.layers import Embedding, LSTM, Dense, Bidirectional, TimeDistributed
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from sklearn.metrics import f1_score
import numpy as np
import pandas as pd
import os
import re

In [47]:
# Load dữ liệu UIT-VSFC

import gdown

# ID folder từ link
folder_id = '1xclbjHHK58zk2X6iqbvMPS2rcy9y9E0X'
folder_url = f'https://drive.google.com/drive/folders/{folder_id}'

# Tải toàn bộ folder về /content/UIT-VSFC
gdown.download_folder(folder_url, quiet=True, use_cookies=False, output='/content/UIT-VSFC')

print("Tải xong! Kiểm tra cấu trúc:")
!ls -la /content/UIT-VSFC
!ls -la /content/UIT-VSFC/train


Tải xong! Kiểm tra cấu trúc:
total 24
drwxr-xr-x 5 root root 4096 Dec  2 14:46 .
drwxr-xr-x 1 root root 4096 Dec  2 14:40 ..
drwxr-xr-x 2 root root 4096 Dec  2 14:46 dev
-rw-r--r-- 1 root root 1084 Apr 28  2020 README.txt
drwxr-xr-x 2 root root 4096 Dec  2 14:46 test
drwxr-xr-x 2 root root 4096 Dec  2 14:46 train
total 936
drwxr-xr-x 2 root root   4096 Dec  2 14:46 .
drwxr-xr-x 5 root root   4096 Dec  2 14:46 ..
-rw-r--r-- 1 root root  22852 Apr 28  2020 sentiments.txt
-rw-r--r-- 1 root root 898090 Apr 28  2020 sents.txt
-rw-r--r-- 1 root root  22852 Apr 28  2020 topics.txt


In [48]:
import os
import pandas as pd
from collections import Counter

# Đường dẫn file
data_dir = "/content/UIT-VSFC"

def load_split(split_name):
  sent_path = os.path.join(data_dir, split_name, "sents.txt")
  senti_path = os.path.join(data_dir, split_name, "sentiments.txt")
  topic_path = os.path.join(data_dir, split_name, "topics.txt")

  # Đọc file
  with open(sent_path, 'r', encoding='utf-8') as f:
      sentences = [line.strip() for line in f if line.strip()]
  with open(senti_path, 'r', encoding='utf-8') as f:
      sentiments = [int(line.strip()) for line in f if line.strip()]
  with open(topic_path, 'r', encoding='utf-8') as f:
      topics = [line.strip() for line in f if line.strip()]

  df = pd.DataFrame({
      'sentence': sentences,
      'sentiment': sentiments,
      'topic': topics
  })
  return df

In [49]:
# Load 3 splits
train_df = load_split("train")
dev_df = load_split("dev")
test_df = load_split("test")

In [50]:
# Tiền xử lý văn bản

def clean_text(text):
    text = text.lower()
    text = re.sub(r'[^\w\s]', '', text)   # Loại bỏ dấu câu
    return text


In [51]:
train_df['sentence'] = train_df['sentence'].apply(clean_text)
dev_df['sentence'] = dev_df['sentence'].apply(clean_text)
test_df['sentence'] = test_df['sentence'].apply(clean_text)

In [52]:
train_df

,sentence,sentiment,topic
0,slide giáo trình đầy đủ,2,1
1,nhiệt tình giảng dạy gần gũi với sinh viên,2,0
2,đi học đầy đủ full điểm chuyên cần,0,1
3,chưa áp dụng công nghệ thông tin và các thiết ...,0,0
4,thầy giảng bài hay có nhiều bài tập ví dụ nga...,2,0
...,...,...,...
11421,chỉ vì môn game mà em học hai lần mà không qua...,0,1
11422,em cảm ơn cô nhiều,2,0
11423,giao bài tập quá nhiều,0,0
11424,giáo viên dạy dễ hiểu nhiệt tình,2,0


In [53]:
# Tokenizer
vocab_size = 20000
tokenizer = Tokenizer(num_words=vocab_size, oov_token="<OOV>")

tokenizer.fit_on_texts(train_df['sentence'])

In [54]:
# Chuyển thành sequnce
train_seq = tokenizer.texts_to_sequences(train_df['sentence'])
dev_seq   = tokenizer.texts_to_sequences(dev_df['sentence'])
test_seq  = tokenizer.texts_to_sequences(test_df['sentence'])

In [55]:
# Padding (post-padding)
max_length = max(len(x) for x in train_seq + dev_seq + test_seq)
print(f"Độ dài tối đa: {max_length}")

train_pad = pad_sequences(train_seq, maxlen=max_length, padding='post', truncating='post')
dev_pad   = pad_sequences(dev_seq,   maxlen=max_length, padding='post', truncating='post')
test_pad  = pad_sequences(test_seq,  maxlen=max_length, padding='post', truncating='post')

Độ dài tối đa: 150


In [56]:
# Labels (0, 1, 2 → one-hot encode)
y_train = to_categorical(train_df['sentiment'], num_classes=3)
y_dev   = to_categorical(dev_df['sentiment'],   num_classes=3)
y_test  = to_categorical(test_df['sentiment'],  num_classes=3)

In [57]:
# Xây dựng mô hình LSTM 5 lớp

embedding_dim = 128

model = Sequential([
    Embedding(input_dim=vocab_size,
              output_dim=embedding_dim,
              mask_zero=True,               # Rất quan trọng với LSTM để bỏ qua padding
              input_length=max_length),

    LSTM(256, return_sequences=True),
    LSTM(256, return_sequences=True),
    LSTM(256, return_sequences=True),
    LSTM(256, return_sequences=True),
    LSTM(256, return_sequences=False),      # Lớp cuối không trả về sequence

    Dense(3, activation='softmax')
])

model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

model.summary()

/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_3 (Embedding)         │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_10 (LSTM)                  │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_11 (LSTM)                  │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_12 (LSTM)                  │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_13 (LSTM)                  │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_14 (LSTM)                  │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [58]:
class F1Callback(tf.keras.callbacks.Callback):
    def __init__(self, dev_data):
        super().__init__()
        self.dev_data = dev_data
        self.best_f1 = 0.0

    def on_epoch_end(self, epoch, logs=None):
        x_dev, y_dev = self.dev_data
        y_pred = np.argmax(self.model.predict(x_dev, verbose=0), axis=1)
        y_true = np.argmax(y_dev, axis=1)
        f1 = f1_score(y_true, y_pred, average='macro')
        print(f" - dev_macro_f1: {f1:.4f}")

        if f1 > self.best_f1:
            self.best_f1 = f1
            self.model.save_weights("best_model.weights.h5")
        print(f" (Best dev F1 so far: {self.best_f1:.4f})")

f1_callback = F1Callback(dev_data=(dev_pad, y_dev))

In [59]:
# Huấn luyện
history = model.fit(
    train_pad, y_train,
    batch_size=64,
    epochs=20,
    validation_data=(dev_pad, y_dev),
    callbacks=[f1_callback],
    verbose=1
)

# Đánh giá cuối cùng trên tập test

# Load trọng số tốt nhất
model.load_weights("/content/best_model.weights.h5")

y_pred_test = np.argmax(model.predict(test_pad), axis=1)
y_true_test = test_df['sentiment'].values

test_f1_macro = f1_score(y_true_test, y_pred_test, average='macro')
test_f1_weighted = f1_score(y_true_test, y_pred_test, average='weighted')

print("KẾT QUẢ CUỐI CÙNG TRÊN TẬP TEST")
print(f"Macro F1-score : {test_f1_macro:.4f}")
print(f"Weighted F1    : {test_f1_weighted:.4f}")

# In thêm F1 từng lớp
from sklearn.metrics import classification_report
print(classification_report(y_true_test, y_pred_test, digits=4))

Epoch 1/20
179/179 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step - accuracy: 0.7101 - loss: 0.6677 - dev_macro_f1: 0.6118
 (Best dev F1 so far: 0.6118)
179/179 ━━━━━━━━━━━━━━━━━━━━ 11s 39ms/step - accuracy: 0.7106 - loss: 0.6667 - val_accuracy: 0.8970 - val_loss: 0.3232
Epoch 2/20
178/179 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step - accuracy: 0.9063 - loss: 0.2854 - dev_macro_f1: 0.6166
 (Best dev F1 so far: 0.6166)
179/179 ━━━━━━━━━━━━━━━━━━━━ 8s 27ms/step - accuracy: 0.9063 - loss: 0.2855 - val_accuracy: 0.9040 - val_loss: 0.2991
Epoch 3/20
177/179 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step - accuracy: 0.9152 - loss: 0.2628 - dev_macro_f1: 0.6184
 (Best dev F1 so far: 0.6184)
179/179 ━━━━━━━━━━━━━━━━━━━━ 5s 29ms/step - accuracy: 0.9152 - loss: 0.2627 - val_accuracy: 0.9065 - val_loss: 0.2979
Epoch 4/20
177/179 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step - accuracy: 0.9226 - loss: 0.2370 - dev_macro_f1: 0.7029
 (Best dev F1 so far: 0.7029)
179/179 ━━━━━━━━━━━━━━━━━━━━ 5s 26ms/step - accuracy: 0.9227 - loss: 0.2370 - val_acc

### Bài 2: Xây dựng mạng GRU gồm 5 lớp với hidden size là 256 cho bài toán phân loại văn bản. Huấn luyện mô hình này trên bộ dữ liệu UIT-VSFC (Vietnamese Student Feedback Corpus) sử dụng Adam làm phương thức tối ưu tham số và đánh giá độ hiệu quả của mô hình sử dụng độ đo F1.

In [60]:
# Xây dựng mô hình GRU 5 lớp (hidden = 256)

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, GRU, Dense
from tensorflow.keras.callbacks import Callback

model_gru = Sequential([
    Embedding(input_dim=vocab_size,
              output_dim=128,
              input_length=max_length,
              mask_zero=True),

    GRU(256, return_sequences=True),
    GRU(256, return_sequences=True),
    GRU(256, return_sequences=True),
    GRU(256, return_sequences=True),
    GRU(256, return_sequences=False),

    Dense(3, activation='softmax')
])

model_gru.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

model_gru.summary()

/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_4 (Embedding)         │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ gru_5 (GRU)                     │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ gru_6 (GRU)                     │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ gru_7 (GRU)                     │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ gru_8 (GRU)                     │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ gru_9 (GRU)                     │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [61]:
# Callback tính macro F1 + lưu model tốt nhất

class F1Callback(Callback):
    def __init__(self, val_data):
        super().__init__()
        self.x_val, self.y_val = val_data
        self.best_f1 = 0.0

    def on_epoch_end(self, epoch, logs=None):
        pred = np.argmax(self.model.predict(self.x_val, verbose=0), axis=1)
        true = np.argmax(self.y_val, axis=1)
        f1 = f1_score(true, pred, average='macro')
        print(f"  dev_macro_f1: {f1:.4f}", end="")
        if f1 > self.best_f1:
            self.best_f1 = f1
            self.model.save_weights("best_gru5_uitvsfc.weights.h5")
            print("Best version was saved")
        else:
            print("")
        print(f"  (Best dev F1 so far: {self.best_f1:.4f})")

f1_cb = F1Callback((dev_pad, y_dev))

In [62]:
# Quá trình huấn luyện

history = model_gru.fit(
    train_pad, y_train,
    batch_size=64,
    epochs=25,
    validation_data=(dev_pad, y_dev),
    callbacks=[f1_cb],
    verbose=1
)

Epoch 1/25
179/179 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step - accuracy: 0.7687 - loss: 0.5518  dev_macro_f1: 0.6127Best version was saved
  (Best dev F1 so far: 0.6127)
179/179 ━━━━━━━━━━━━━━━━━━━━ 10s 32ms/step - accuracy: 0.7692 - loss: 0.5510 - val_accuracy: 0.8983 - val_loss: 0.3139
Epoch 2/25
179/179 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step - accuracy: 0.9133 - loss: 0.2729  dev_macro_f1: 0.6205Best version was saved
  (Best dev F1 so far: 0.6205)
179/179 ━━━━━━━━━━━━━━━━━━━━ 5s 26ms/step - accuracy: 0.9133 - loss: 0.2729 - val_accuracy: 0.9090 - val_loss: 0.2798
Epoch 3/25
177/179 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step - accuracy: 0.9297 - loss: 0.2313  dev_macro_f1: 0.7107Best version was saved
  (Best dev F1 so far: 0.7107)
179/179 ━━━━━━━━━━━━━━━━━━━━ 4s 24ms/step - accuracy: 0.9297 - loss: 0.2313 - val_accuracy: 0.9141 - val_loss: 0.2721
Epoch 4/25
178/179 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step - accuracy: 0.9341 - loss: 0.1968  dev_macro_f1: 0.7484Best version was saved
  (Best dev F1 so far: 0.748

In [63]:
test_df

,sentence,sentiment,topic
0,nói tiếng anh lưu loát,2,0
1,giáo viên rất vui tính,2,0
2,cô max có tâm,2,0
3,giảng bài thu hút dí dỏm,2,0
4,giáo viên không giảng dạy kiến thức hướng dẫn...,0,0
...,...,...,...
3161,các slide khó hiểu ngôn ngữ trong slide phức ...,0,0
3162,giáo viên giảng dạy có tâm huyết,2,0
3163,chia sẻ cho em nhiều điều hay,2,0
3164,em tiếp thu chậm,0,0


In [64]:
model_gru.load_weights("/content/best_gru5_uitvsfc.weights.h5")

pred_test = np.argmax(model_gru.predict(test_pad, verbose=0), axis=1)
true_test = test_df["sentiment"].values

macro_f1 = f1_score(true_test, pred_test, average='macro')
weighted_f1 = f1_score(true_test, pred_test, average='weighted')

print(f"Optimizer       : Adam (lr=0.001)")
print(f"Test Macro F1   : {macro_f1:.4f}")
print(f"Test Weighted F1: {weighted_f1:.4f}")

print(classification_report(true_test, pred_test,
                      target_names=['Tiêu cực', 'Trung lập', 'Tích cực'],
                      digits=4))

Optimizer       : Adam (lr=0.001)
Test Macro F1   : 0.7691
Test Weighted F1: 0.8955
              precision    recall  f1-score   support

    Tiêu cực     0.9002    0.9283    0.9140      1409
   Trung lập     0.5433    0.4132    0.4694       167
    Tích cực     0.9250    0.9226    0.9238      1590

    accuracy                         0.8983      3166
   macro avg     0.7895    0.7547    0.7691      3166
weighted avg     0.8938    0.8983    0.8955      3166



### Bài 3: Xây dựng kiến trúc Encoder trong đó Encoder gồm 5 lớp BiLSTM với hidden size là 256 cho bài toán nhận diện thực thể (Name Entity Recognition). Huấn luyện mô hình trên bộ dữ liệu PhoNER và đánh giá độ hiệu quả của mô hình sử dụng độ đo F1.

In [65]:
!git clone https://github.com/VinAIResearch/PhoNER_COVID19.git
%cd PhoNER_COVID19
!ls

Cloning into 'PhoNER_COVID19'...
remote: Enumerating objects: 61, done.
remote: Counting objects: 100% (61/61), done.
remote: Compressing objects: 100% (42/42), done.
remote: Total 61 (delta 24), reused 41 (delta 17), pack-reused 0 (from 0)
Receiving objects: 100% (61/61), 3.61 MiB | 15.94 MiB/s, done.
Resolving deltas: 100% (24/24), done.
/content/PhoNER_COVID19/PhoNER_COVID19
AnnotationGuidelinesforPhoNER_COVID19_VietnameseVersion.pdf  data  README.md


In [66]:
import json
from pathlib import Path
import pandas as pd
data_dir = "/content/PhoNER_COVID19/data/word"

def load_phoner_jsonl(file_path):
    data = []
    with open(file_path, 'r', encoding='utf-8') as f:
        for line in f:
            line = line.strip()
            if line:
                item = json.loads(line)
                data.append({
                    "tokens":   item["words"],      # tên cột: "words"
                    "ner_tags": item["tags"]        # tên cột: "tags"
                })
    return pd.DataFrame(data)

# LOAD
train_df = load_phoner_jsonl(data_dir + "/train_word.json")
dev_df   = load_phoner_jsonl(data_dir + "/dev_word.json")
test_df  = load_phoner_jsonl(data_dir + "/test_word.json")



In [67]:
train_df.head()

,tokens,ner_tags
0,"[Đồng_thời, ,, bệnh_viện, tiếp_tục, thực_hiện,...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
1,"["", Số, bệnh_viện, có_thể, tiếp_nhận, bệnh_nhâ...","[O, O, O, O, O, O, O, B-SYMPTOM_AND_DISEASE, I..."
2,"[Ngoài_ra, ,, những, người, tiếp_xúc, gián_tiế...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
3,"[Bà, này, khi, trở, về, quá_cảnh, Doha, (, Qat...","[O, O, O, O, O, O, B-LOCATION, O, B-LOCATION, ..."
4,"["", Bệnh_nhân, 523, "", và, chồng, là, "", bệnh_...","[O, O, B-PATIENT_ID, O, O, O, O, O, O, B-PATIE..."


In [68]:
# In ra các entity

all_tags = [tag for tags in train_df['ner_tags'] for tag in tags]
tag2id = {tag: idx for idx, tag in enumerate(sorted(set(all_tags)))}
id2tag = {idx: tag for tag, idx in tag2id.items()}

print("Tags:", tag2id)
num_tags = len(tag2id)
print("Số lớp NER:", num_tags)

Tags: {'B-AGE': 0, 'B-DATE': 1, 'B-GENDER': 2, 'B-JOB': 3, 'B-LOCATION': 4, 'B-NAME': 5, 'B-ORGANIZATION': 6, 'B-PATIENT_ID': 7, 'B-SYMPTOM_AND_DISEASE': 8, 'B-TRANSPORTATION': 9, 'I-AGE': 10, 'I-DATE': 11, 'I-JOB': 12, 'I-LOCATION': 13, 'I-NAME': 14, 'I-ORGANIZATION': 15, 'I-PATIENT_ID': 16, 'I-SYMPTOM_AND_DISEASE': 17, 'I-TRANSPORTATION': 18, 'O': 19}
Số lớp NER: 20


In [69]:
# Tokenizer + Padding

VOCAB_SIZE = 20000
tokenizer = Tokenizer(num_words=VOCAB_SIZE, oov_token="<OOV>", lower=True)
tokenizer.fit_on_texts(train_df['tokens'])

def encode_data(df):
    seq = tokenizer.texts_to_sequences(df['tokens'])
    seq_pad = pad_sequences(seq, padding='post')
    tags_pad = pad_sequences(df['ner_tags'].apply(lambda x: [tag2id[t] for t in x]),
                             padding='post', value=tag2id['O'])
    return seq_pad, tags_pad

X_train, y_train = encode_data(train_df)
X_dev,   y_dev   = encode_data(dev_df)
X_test,  y_test  = encode_data(test_df)

max_len = X_train.shape[1]
print("Max length =", max_len)

Max length = 161


In [70]:
# Xây dựng mô hình: 5 lớp BiLSTM (hidden=256)

model = Sequential([
    Embedding(input_dim=VOCAB_SIZE,
              output_dim=256,
              input_length=max_len,
              mask_zero=True),

    Bidirectional(LSTM(256, return_sequences=True)),
    Bidirectional(LSTM(256, return_sequences=True)),
    Bidirectional(LSTM(256, return_sequences=True)),
    Bidirectional(LSTM(256, return_sequences=True)),
    Bidirectional(LSTM(256, return_sequences=True)),  # lớp 5

    TimeDistributed(Dense(num_tags, activation='softmax'))
])

model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    loss='sparse_categorical_crossentropy',
    metrics =['accuracy']
)

/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [71]:
!pip install seqeval

In [72]:
from seqeval.metrics import classification_report as seqeval_report

class NERF1Callback(tf.keras.callbacks.Callback):
    def __init__(self, val_data):
        super().__init__()
        self.X_val, self.y_val = val_data
        self.best_f1 = 0.0

    def on_epoch_end(self, epoch, logs=None):
        pred_ids = np.argmax(self.model.predict(self.X_val, verbose=0), axis=-1)
        pred_tags = [[id2tag[idx] for idx in seq if idx in id2tag] for seq in pred_ids]
        true_tags = [[id2tag[idx] for idx in seq if idx in id2tag] for seq in self.y_val]

        # Chỉ lấy đến độ dài thật (loại bỏ padding)
        pred_tags = [p[:len(t)] for p, t in zip(pred_tags, true_tags)]

        from seqeval.metrics import f1_score
        f1 = f1_score(true_tags, pred_tags)
        print(f"  dev_entity_f1: {f1:.4f}", end="")
        if f1 > self.best_f1:
            self.best_f1 = f1
            self.model.save_weights("best_bilstm5_phoner.weights.h5")
            print("  ← NEW BEST!")
        else:
            print("")
        print(f"  (Best dev F1 = {self.best_f1:.4f})")

f1_cb = NERF1Callback((X_dev, y_dev))

In [73]:
# Training

history = model.fit(
    X_train, y_train,
    batch_size=32,
    epochs=30,
    validation_data=(X_dev, y_dev),
    callbacks=[f1_cb],
    verbose=1
)

Epoch 1/30
158/158 ━━━━━━━━━━━━━━━━━━━━ 0s 90ms/step - accuracy: 0.9311 - loss: 1.2073  dev_entity_f1: 0.0127  ← NEW BEST!
  (Best dev F1 = 0.0127)
158/158 ━━━━━━━━━━━━━━━━━━━━ 46s 154ms/step - accuracy: 0.9313 - loss: 1.2061 - val_accuracy: 0.9503 - val_loss: 0.9652
Epoch 2/30
157/158 ━━━━━━━━━━━━━━━━━━━━ 0s 91ms/step - accuracy: 0.9701 - loss: 0.7093  dev_entity_f1: 0.6019  ← NEW BEST!
  (Best dev F1 = 0.6019)
158/158 ━━━━━━━━━━━━━━━━━━━━ 18s 116ms/step - accuracy: 0.9702 - loss: 0.7074 - val_accuracy: 0.9798 - val_loss: 0.3548
Epoch 3/30
157/158 ━━━━━━━━━━━━━━━━━━━━ 0s 89ms/step - accuracy: 0.9903 - loss: 0.2041  dev_entity_f1: 0.7871  ← NEW BEST!
  (Best dev F1 = 0.7871)
158/158 ━━━━━━━━━━━━━━━━━━━━ 19s 123ms/step - accuracy: 0.9903 - loss: 0.2037 - val_accuracy: 0.9881 - val_loss: 0.2132
Epoch 4/30
157/158 ━━━━━━━━━━━━━━━━━━━━ 0s 89ms/step - accuracy: 0.9953 - loss: 0.0994  dev_entity_f1: 0.8320  ← NEW BEST!
  (Best dev F1 = 0.8320)
158/158 ━━━━━━━━━━━━━━━━━━━━ 18s 113ms/step - ac

In [74]:
# Đánh giá test - Entity F1
model.load_weights("/content/best_bilstm5_phoner.weights.h5")

pred_test_ids = np.argmax(model.predict(X_test, verbose=0), axis=-1)

pred_test_tags = []
true_test_tags = []

for i in range(len(test_df)):
    length = sum(X_test[i] != 0)  # độ dài thật
    pred_seq = [id2tag[idx] for idx in pred_test_ids[i][:length]]
    true_seq = [id2tag[idx] for idx in y_test[i][:length]]
    pred_test_tags.append(pred_seq)
    true_test_tags.append(true_seq)

from seqeval.metrics import classification_report, f1_score

final_f1 = f1_score(true_test_tags, pred_test_tags)



print(f"Optimizer       : Adam (lr=0.001)")
print(f"Test Entity F1  : {final_f1:.4f}")

print(classification_report(true_test_tags, pred_test_tags, digits=4))

Optimizer       : Adam (lr=0.001)
Test Entity F1  : 0.8723
                     precision    recall  f1-score   support

                AGE     0.8744    0.9330    0.9027       582
               DATE     0.9422    0.9565    0.9493      1654
             GENDER     0.9443    0.9177    0.9308       462
                JOB     0.5714    0.5087    0.5382       173
           LOCATION     0.8895    0.8631    0.8761      4441
               NAME     0.8454    0.5157    0.6406       318
       ORGANIZATION     0.8228    0.7108    0.7627       771
         PATIENT_ID     0.9229    0.9546    0.9385      2005
SYMPTOM_AND_DISEASE     0.8218    0.7306    0.7735      1136
     TRANSPORTATION     0.7688    0.6891    0.7268       193

          micro avg     0.8879    0.8572    0.8723     11735
          macro avg     0.8404    0.7780    0.8039     11735
       weighted avg     0.8852    0.8572    0.8694     11735

